# Load torch device

In [1]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, AdamW

from Datasets.batching import BatchManager

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# Model Initialization

In [2]:
from Models.CleanEvGNN.recognition import RecognitionModel as EvGNN
from torch_geometric.data import Data as PyGData
from Datasets.ncaltech101 import NCaltech
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_size: tuple[int, int] = NCaltech.get_info().image_size  # (height=240, width=180)
input_shape: tuple[int, int, int] = (*image_size, 3)

# Vertausche Dimensionen: pos ist [x, y] = [width, height], also übergeben wir (width, height)
img_shape_for_model = (image_size[1], image_size[0])  # (180, 240) statt (240, 180)

evgnn = EvGNN(
    network="graph_res",
    dataset="ncaltech101",
    num_classes = len(NCaltech.get_info().classes),
    img_shape=img_shape_for_model,  # (width=180, height=240)
    dim=3,
    conv_type="fuse",
    distill=False,        # <– no KD, just normal training
).to(device)


def transform_graph(graph: PyGData) -> PyGData:

    graph = evgnn.data_transform(
        graph, n_samples = 25000, sampling = True,
        beta =  0.5e-5, radius = 5.0,
        max_neighbors = 32
    ).to(device)
    # Stelle sicher, dass Positionen im Bildbereich [0, image_size] liegen
    graph.pos[:, 0] = torch.clamp(graph.pos[:, 0], 0, image_size[1] - 1)  # x: 0-179
    graph.pos[:, 1] = torch.clamp(graph.pos[:, 1], 0, image_size[0] - 1)  # y
    return graph

[GraphRes.__init__] input_shape[:2]=tensor([180, 240]), pooling_dm_dims=tensor([16., 16.])
[GraphRes.__init__] num_grids_per_dim=tensor([12, 15], dtype=torch.int32), num_grids=180


# Dataset Initialization and processing (from the parsed dataset from the aegnn issues thread)

In [3]:
#Instantiating the ncaltech dataset
ncaltech = NCaltech(
    root=r"C:\Users\hanne\Documents\Hannes\Uni\Maastricht\Project\Datasets",
    transform=transform_graph
)

# Processing the training part of the dataset
ncaltech.process(modes = ["training"])

x

📂 Processing folder: accordion


accordion:   0%|          | 0/55 [00:00<?, ?it/s]


📂 Processing folder: airplanes


airplanes:   0%|          | 0/800 [00:00<?, ?it/s]


📂 Processing folder: anchor


...

windsor_chair:   0%|          | 0/56 [00:00<?, ?it/s]


📂 Processing folder: wrench


wrench:   0%|          | 0/39 [00:00<?, ?it/s]


📂 Processing folder: yin_yang


yin_yang:   0%|          | 0/60 [00:00<?, ?it/s]

Display example events data point

In [4]:
training_set = BatchManager(
    dataset=ncaltech,
    batch_size=8,
    mode="training"
)

In [5]:
optimizer = Adam(evgnn.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()

classes = ncaltech.get_info().classes

cls_to_idx = dict(zip(classes, range(len(classes))))

In [6]:
evgnn.train()
for i in range(200):
    examples = next(training_set)
    reference = torch.tensor([cls_to_idx[cls] for cls in examples.label], dtype=torch.long).to(device)
    out = evgnn(examples)
    loss = loss_fn(out, reference)
    loss.backward()
    optimizer.step()
    print(f"Iteration {i} loss: {loss.item()}")
    optimizer.zero_grad()

torch.save(evgnn.state_dict(), "aegnn_ncaltech_test.pth")

[data_transform] Start - pos shape: torch.Size([86925, 3])
  x range: [0.00, 232.00]
  y range: [0.00, 142.00]
  t range: [57.00, 298510.00]
  Expected bounds: x=[0, 239], y=[0, 179]
[data_transform] Nach Clamping:
  x range: [0.00, 232.00]
  y range: [0.00, 142.00]
[data_transform] Nach Sampling - pos shape: torch.Size([25000, 3])
[data_transform] Ende - edge_index shape: torch.Size([2, 776324])
[data_transform] Start - pos shape: torch.Size([31166, 3])
  x range: [0.00, 118.00]
  y range: [0.00, 172.00]
  t range: [5.00, 300325.00]
  Expected bounds: x=[0, 239], y=[0, 179]
[data_transform] Nach Clamping:
  x range: [0.00, 118.00]
  y range: [0.00, 172.00]
[data_transform] Nach Sampling - pos shape: torch.Size([25000, 3])
[data_transform] Ende - edge_index shape: torch.Size([2, 788927])
[data_transform] Start - pos shape: torch.Size([126026, 3])
  x range: [0.00, 232.00]
  y range: [0.00, 88.00]
  t range: [30.00, 296427.00]
  Expected bounds: x=[0, 239], y=[0, 179]
[data_transform] N

KeyboardInterrupt: 